In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('company_nps_data.csv',encoding='cp949')

In [6]:
data.head()

,회사ID,연매출액,년도,월,월별_연금보험료,월별_직원수
0,233757,41688077,2018,7,8912380,36
1,123464,1760695,2018,3,942380,5
2,342124,3221341,2018,7,781180,4
3,386420,4815584,2015,11,3795900,14
4,129243,9799736,2018,10,40724680,151


In [3]:
'''
data['년도'] = data['년도'].astype(str)
data['월'] = data['월'].astype(str)
data['월'] = data['월'].str.zfill(2)

data['연월'] = data['년도'] + data['월']
'''

In [ ]:
# Nan 값 없음
data.isna().sum()

회사ID        0
연매출액        0
년도          0
월           0
월별_연금보험료    0
월별_직원수      0
연월          0
dtype: int64

36개월 이상 데이터가 존재하는 기업


In [ ]:
'''
3년 이상된 기업

from collections import Counter

years = data['년도'].unique()
company = []

for year in years:
  company += list(set(data[data['년도']==year].iloc[:,0]))

company = Counter(company).most_common()
under_3 = [c[0]for c in company if c[1]<3]

df = data[~data['회사ID'].isin(under_3)]
'''

In [3]:
df = data.groupby('회사ID').filter(lambda x: len(x)>=36) 

직원수 0 기업 제외 => 3년 이상인 기업에서는 없음 5인 미만 영세기업 제외는 어떨까? 

In [94]:
# 0곳 해당
temp = df[df['월별_직원수']==0].iloc[:,0].tolist()

In [95]:
len(set(temp))

0

연매출액 249억 이상 기업 (연매출 단위: 천원)

In [4]:
# 289개
temp = df[df['연매출액']>=24900000].iloc[:,0].tolist()
df = df[df['회사ID'].isin(temp)]

보험료, 직원수, 연매출 증가율 칼럼 추가

In [7]:
# 회사ID, 년도, 월 순으로 정렬 후 인덱스 다시 매김
df = df.sort_values(by=['회사ID','년도','월'])
df = df.reset_index(drop=True)

In [6]:
worker = pd.DataFrame

In [8]:
worker = df[['회사ID','년도','월별_직원수','연매출액']].groupby(['회사ID','년도']).mean()
worker['전년도_평균직원수'] = worker.groupby('회사ID')['월별_직원수'].shift(1)
worker['연평균_직원수_증감률'] = ( (worker['월별_직원수'] - worker['전년도_평균직원수']) / worker['전년도_평균직원수'] ) * 100
worker['1인_평균_연매출'] = worker['연매출액'] / worker['월별_직원수']
worker['전년도_1인_평균_연매출'] = worker.groupby('회사ID')['1인_평균_연매출'].shift(1)
worker['1인_평균_연매출_증감률'] = ( (worker['1인_평균_연매출'] - worker['전년도_1인_평균_연매출']) / worker['전년도_1인_평균_연매출'] ) * 100

In [9]:
worker

월별_직원수         연매출액  ...  전년도_1인_평균_연매출  1인_평균_연매출_증감률
회사ID   년도                             ...                              
122968 2015   88.000000   37415377.0  ...            NaN            NaN
       2016   76.166667   26858719.0  ...  425174.738636     -17.062127
       2017   71.250000   32997924.0  ...  352630.884026      31.335280
       2018   66.500000   31514468.0  ...  463128.757895       2.326139
125878 2016   26.833333   23932243.0  ...            NaN            NaN
...                 ...          ...  ...            ...            ...
510329 2017  593.000000  134718002.0  ...  228280.204942      -0.481760
       2018  582.083333  143513683.0  ...  227180.441821       8.526852
510449 2016   63.833333   24450163.0  ...            NaN            NaN
       2017   87.916667   25154072.0  ...  383031.274151     -25.303053
       2019  145.333333   28299495.0  ...  286112.667299     -31.942442

[1132 rows x 7 columns]

In [10]:
worker = worker.reset_index()

In [11]:
temp = worker[worker['연평균_직원수_증감률']<0].iloc[:,0].tolist()
worker = worker[~worker['회사ID'].isin(temp)]

temp = worker[worker['1인_평균_연매출_증감률']<0].iloc[:,0].tolist()
worker = worker[~worker['회사ID'].isin(temp)]

In [12]:
test = worker[['회사ID','1인_평균_연매출_증감률']].groupby('회사ID').mean()

In [13]:
test = test.reset_index()
temp = test.sort_values(by='1인_평균_연매출_증감률', ascending=False).head(10).iloc[:,0].tolist()

In [15]:
temp

[133493,
 127366,
 420165,
 404804,
 306768,
 128405,
 132335,
 420304,
 294337,
 318919]

In [20]:
topten = df[df['회사ID'].isin(temp)]

In [21]:
topten

,회사ID,연매출액,년도,월,월별_연금보험료,월별_직원수
682,127366,55796793,2015,11,78792380,244
683,127366,55796793,2015,12,79377740,245
684,127366,74306879,2016,1,79088940,244
685,127366,74306879,2016,2,80368380,248
686,127366,74306879,2016,3,81198360,251
...,...,...,...,...,...,...
9972,420304,98205039,2018,8,52290720,130
9973,420304,98205039,2018,9,51869520,129
9974,420304,98205039,2018,10,51361020,127
9975,420304,98205039,2018,11,50939820,126
